In [8]:
%pip install langchain openai faiss-cpu tiktoken pypdf langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00


In [ ]:
import os
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

# Load medical documents
docs_path = "../docs"
documents = []

for file in os.listdir(docs_path):
    file_path = os.path.join(docs_path, file)

    # Load PDFs with PyPDFLoader
    if file.endswith(".pdf"):
        loader = PyPDFLoader(file_path)
        documents.extend(loader.load())

    # Load text files with TextLoader
    elif file.endswith(".txt"):
        loader = TextLoader(file_path)
        documents.extend(loader.load())

# Split into smaller chunks for better retrieval
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

print(f"Loaded {len(chunks)} document chunks successfully!")


In [4]:
%pip install jq

   ---------------------------------------- 0.0/420.0 kB ? eta -:--:--
    --------------------------------------- 10.2/420.0 kB ? eta -:--:--
    --------------------------------------- 10.2/420.0 kB ? eta -:--:--
   - -------------------------------------- 20.5/420.0 kB 72.6 kB/s eta 0:00:06
   -- ------------------------------------ 30.7/420.0 kB 145.2 kB/s eta 0:00:03
   -- ------------------------------------ 30.7/420.0 kB 145.2 kB/s eta 0:00:03
   ----- --------------------------------- 61.4/420.0 kB 192.5 kB/s eta 0:00:02
   ----- --------------------------------- 61.4/420.0 kB 192.5 kB/s eta 0:00:02
   ----- --------------------------------- 61.4/420.0 kB 192.5 kB/s eta 0:00:02
   ------ -------------------------------- 71.7/420.0 kB 157.1 kB/s eta 0:00:03
   -------- ------------------------------ 92.2/420.0 kB 174.7 kB/s eta 0:00:02
   -------- ------------------------------ 92.2/420.0 kB 174.7 kB/s eta 0:00:02
   ------------- ------------------------ 153.6/420.0 kB 248.0 kB


[notice] A new release of pip is available: 24.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from langchain.document_loaders import JSONLoader
from langchain.text_splitter import CharacterTextSplitter
import os

# Initialize the text splitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# Load JSON documents
kb_chunks_path = "../docs/kb_chunks"
kb_documents = []

for file in os.listdir(kb_chunks_path):
    file_path = os.path.join(kb_chunks_path, file)

    if file.endswith(".json"):
        loader = JSONLoader(
            file_path=file_path,
            jq_schema=".content",  # <- récupère uniquement le champ "content"
            text_content=False     # <- retourne des objets Document, pas juste le texte brut
        )
        kb_documents.extend(loader.load())

# Split JSON documents into smaller chunks
chunks = text_splitter.split_documents(kb_documents)

print(f"Loaded {len(chunks)} JSON document chunks successfully!")


Loaded 447 JSON document chunks successfully!


In [13]:
!pip install -q sentence-transformers pymilvus


In [7]:
from pymilvus import MilvusClient, DataType

CLUSTER_ENDPOINT = "https://in03-22706440ab83439.serverless.gcp-us-west1.cloud.zilliz.com"
TOKEN = "fd3df03a845da1e42df2a2975f1d4733f62b044415d021f9f6047ef1f631504509c1a870e1370ebc8587c04883fae0c72c886204"

# 1. Set up a Milvus client
client = MilvusClient(
    uri=CLUSTER_ENDPOINT,
    token=TOKEN
)

In [8]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Milvus
from pymilvus import Collection, FieldSchema, CollectionSchema, DataType, connections, utility

# Define the collection name
collection_name = "document_embeddings"

# Connect to Milvus (ensure you've set up the connection correctly)
connections.connect("default", uri=CLUSTER_ENDPOINT, token=TOKEN)

# # List all collections in the connected Milvus instance
# existing_collections = utility.list_collections()

# # Check if the collection exists or create it
# if collection_name not in existing_collections:
#     # Define the schema and create the collection
#     embedding_dim = 768  # Example dimension, make sure this matches your model

#     fields = [
#         FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=embedding_dim),
#         FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
#     ]
#     schema = CollectionSchema(fields, description="Document embeddings for retrieval")

#     # Create the collection
#     collection = Collection(name=collection_name, schema=schema)
# else:
#     collection = Collection(collection_name)

# print(f"Collection '{collection_name}' is ready.")


In [19]:
from pymilvus import Collection

# Load the collection
collection = Collection(name=collection_name)

# Drop the collection
collection.drop()
print(f"Collection '{collection_name}' has been deleted.")

Collection 'document_embeddings' has been deleted.


In [ ]:
# from pymilvus import connections

# # Disconnect from all existing connections
# connections.disconnect("default")

# # Now try to connect again
# connections.connect("default", uri=CLUSTER_ENDPOINT, token=TOKEN)
# print("Successfully connected to Milvus!")


Successfully connected to Milvus!


In [9]:
# Load your local embeddings model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


C:\Users\weszi\AppData\Local\Temp\ipykernel_3864\2060263944.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\weszi\AppData\Local\Programs\Python\Python38\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [15]:
from pymilvus import Collection

collection = Collection(name=collection_name)
print(collection.schema)


{'auto_id': True, 'description': '', 'fields': [{'name': 'source', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 65535}}, {'name': 'page', 'description': '', 'type': <DataType.INT64: 5>}, {'name': 'text', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 65535}}, {'name': 'pk', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': True}, {'name': 'vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 384}}], 'enable_dynamic_field': False}


In [20]:
from langchain.vectorstores import Milvus

# Prépare les données selon le schéma
texts = [doc.page_content for doc in chunks]
metadatas = [doc.metadata for doc in chunks]
embs = embeddings.embed_documents(texts)  # attention : bien utiliser la même dimension que `vector`

# Prépare chaque champ du schéma
sources = [meta.get("source", "") for meta in metadatas]
pages = [meta.get("page", 0) for meta in metadatas]

# Structure finale (doit contenir 4 listes dans cet ordre)
docs_to_insert = {
    "source": sources,
    "page": pages,
    "text": texts,
    "vector": embs
}

# Crée une instance de Milvus connectée à ta collection
vector_store = Milvus(
    embedding_function=embeddings,
    collection_name=collection_name,
    connection_args={
        "uri": CLUSTER_ENDPOINT,
        "token": TOKEN,
    }
)

# Crée une liste d'ids uniques (par exemple, utilise un range pour chaque document)
ids = [str(i) for i in range(len(texts))]  # IDs uniques sous forme de string

# Conversion des pages en int pour correspondre au schéma Milvus
for meta in metadatas:
    if "page" in meta:
        try:
            meta["page"] = int(meta["page"])
        except ValueError:
            meta["page"] = 0

# Ajoute les documents en fournissant les ids
vector_store.add_texts(
    texts=texts,
    metadatas=metadatas,
    ids=ids
)


['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63',
 '64',
 '65',
 '66',
 '67',
 '68',
 '69',
 '70',
 '71',
 '72',
 '73',
 '74',
 '75',
 '76',
 '77',
 '78',
 '79',
 '80',
 '81',
 '82',
 '83',
 '84',
 '85',
 '86',
 '87',
 '88',
 '89',
 '90',
 '91',
 '92',
 '93',
 '94',
 '95',
 '96',
 '97',
 '98',
 '99',
 '100',
 '101',
 '102',
 '103',
 '104',
 '105',
 '106',
 '107',
 '108',
 '109',
 '110',
 '111',
 '112',
 '113',
 '114',
 '115',
 '116',
 '117',
 '118',
 '119',
 '120',
 '121',
 '122',
 '123',
 '124',
 '125',
 '126',
 '127',
 '128',
 '129',
 '130',
 '131',
 '132',
 '133',
 '134',
 '135',
 '136',
 '137',
 '138'

In [13]:
# Store the document embeddings into Milvus
try:
    vector_store = Milvus.from_documents(
        chunks,
        embeddings,
        collection_name=collection_name,
        connection_args={
            "uri": CLUSTER_ENDPOINT,
            "token": TOKEN,  # omit if unused
        }
    )
except Exception as e:
    print(f"❌ Failed to connect to Milvus cluster: {e}")

Failed to insert batch starting at entity: 0/447


❌ Failed to connect to Milvus cluster: <DataNotMatchException: (code=1, message=The data doesn't match with schema fields, expect 4 list, got 3)>


In [11]:
query = "What are the symptoms of asthma?"
retrieved_docs = vector_store.similarity_search(query, k=3)

print("Top Retrieved Documents:")
for doc in retrieved_docs:
    # print the source
    print(f"Source: {doc.metadata['source']}")
    # print the first 300 characters of the page content
    print(doc.page_content[:300])  # Print a snippet
    print("\n")


Top Retrieved Documents:
Source: ../docs\3.pdf
Home  Health  Conditions and Diseases
Signs of Respiratory
Distress
Signs of Respiratory Distress
Learning the signs of respiratory
distress
People having trouble breathing often show signs that they are having to work
harder to breathe or are not getting enough oxygen, indicating respiratory
dist


Source: ../docs\2.pdf
Global Regions
Donate
Home / Health topics / Chronic respiratory diseases
CreditsChronic respiratory diseases
Chronic respiratory diseases (CRDs) a ect the airways and other structures of the lungs. Some of the most
common are chronic obstructive pulmonary disease (COPD), asthma, occupational lung d


Source: ../docs\4.pdf
For Everyone 
Situation Summary
About Respiratory Illnesses
Risk Factors
Prevention
Treatment
Data and Trends
Resources to Prepare for Flu, COVID-19, and RSV
Respiratory Virus Guidance
Health Care Providers 
Best Practices for Patient Care
Talking with Patients
  Sign up for Email Updates  
ESPA


